# Colour - Spectral Signal

In [1]:
import numpy as np

import colour

INTERPOLATORS = colour.CaseInsensitiveMapping({
    'CubicSpline': colour.CubicSplineInterpolator,
    'Linear': colour.LinearInterpolator,
    'Pchip': colour.PchipInterpolator,
    'Sprague': colour.SpragueInterpolator})


class ContinuousSignal(object):
    def __init__(self,
                 data,
                 index=None,
                 interpolation_method='Sprague',
                 extrapolation_method='Constant',
                 extrapolation_left=np.nan,
                 extrapolation_right=np.nan):
        self.__values = data  # Handle unpacking of dicts, arrays, etc...
        self.__index = index

        self.__interpolation_method = None
        self.__extrapolation_method = None
        self.__extrapolation_left = None
        self.__extrapolation_right = None
        self.interpolation_method = interpolation_method
        self.extrapolation_method = extrapolation_method
        self.extrapolation_left = extrapolation_left
        self.extrapolation_right = extrapolation_right

        self.__create_function()

    @property
    def values(self):
        return self.__values

    @values.setter
    def values(self, value):
        self.__values = value
        self.__create_function()

    @property
    def index(self):
        return self.__index

    @index.setter
    def index(self, value):
        self.__index = value
        self.__create_function()

    @property
    def interpolation_method(self):
        return self.__interpolation_method

    @interpolation_method.setter
    def interpolation_method(self, value):
        self.__interpolation_method = value
        self.__create_function()

    @property
    def extrapolation_method(self):
        return self.__extrapolation_method

    @extrapolation_method.setter
    def extrapolation_method(self, value):
        self.__extrapolation_method = value
        self.__create_function()

    @property
    def extrapolation_left(self):
        return self.__extrapolation_left

    @extrapolation_left.setter
    def extrapolation_left(self, value):
        self.__extrapolation_left = value
        self.__create_function()

    @property
    def extrapolation_right(self):
        return self.__extrapolation_right

    @extrapolation_right.setter
    def extrapolation_right(self, value):
        self.__extrapolation_right = value
        self.__create_function()

    @property
    def function(self):
        return self.__function

    @index.setter
    def function(self, value):
        raise AttributeError(
            '"{0}" attribute is read only!'.format('function'))

    def __create_function(self):
        self.__function = colour.Extrapolator1d(
            INTERPOLATORS[self.__interpolation_method](
                self.__index, self.__values),
            method=self.__extrapolation_method,
            left=self.__extrapolation_left,
            right=self.__extrapolation_right)

    def __getitem__(self, x):
        if type(x) is slice:
            return self.__values[x]
        else:
            return self.__function(x)

    def __setitem__(self, x, value):
        if type(x) is slice:
            self.__values[x] = value
        else:
            in_index = np.in1d(x, index)
            print(in_index)
            indexes = np.searchsorted(self.__index, x)
            self.__index = np.insert(self.__index, indexes, x)
            self.__values = np.insert(self.__values, indexes, value)

        self.__create_function

    def __add__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)
        
        self.__values += x

        return self

    def __sub__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)
        
        self.__values -= x

        return self

    def __mul__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)
        
        self.__values *= x

        return self

    def __div__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)
        
        self.__values /= x

        return self
    
    __truediv__ = __div__

index = np.arange(0, 1000, 100)
values = np.linspace(1, 10, index.size)
cs1 = ContinuousSignal(values, index)

colour.message_box('Item Operations')

print('cs1')
print(cs1.values)
print(cs1.index)

print('\n')

exp = 'cs1[150.25]'
print(exp)
print(eval(exp))

print('\n')

print('cs1[np.linspace(100, 400, 10)]')
print(cs1[np.linspace(100, 400, 10)])

print('\n')

print('cs1[0:3]')
print(cs1[0:3])

print('\n')

print('cs1[10] = np.pi')
cs1[10] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[(200, 300)] = np.pi')
cs1[(200, 300)] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[(0, 850)] = np.pi')
cs1[(0, 850)] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[0:9] = np.pi')
cs1[0:9] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

colour.message_box('Arithmetical Operations with Mismatching Index')

cs1 = ContinuousSignal(values, index)
cs2 = ContinuousSignal(values, index + 400)

print('cs1')
print(cs1.values)
print(cs1.index)

print('cs2')
print(cs2.values)
print(cs2.index)

print('\n')

print('cs1 += cs2')
cs1 += cs2
print(cs1.values)

print('\n')

print('cs1 += 1')
cs1 += 1
print(cs1.values)

print('\n')

print('cs1 -= np.ones(index.size)')
cs1 -= np.ones(index.size)
print(cs1.values)

*                                                                             *
*   Item Operations                                                           *
*                                                                             *
cs1
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


cs1[150.25]
2.5025


cs1[np.linspace(100, 400, 10)]
[ 2.          2.33333333  2.66666667  3.          3.33333333  3.66666667
  4.          4.33333333  4.66666667  5.        ]


cs1[0:3]
[ 1.  2.  3.]


cs1[10] = np.pi
[False]
[  1.           3.14159265   2.           3.           4.           5.           6.
   7.           8.           9.          10.        ]
[  0  10 100 200 300 400 500 600 700 800 900]


cs1[(200, 300)] = np.pi
[ True  True]
[  1.           3.14159265   2.           3.14159265   3.           3.14159265
   4.           5.           6.           7.           8.           9.          10.        ]
[  0  10 100 200 200 300 300 400 500 6